<a href="https://colab.research.google.com/github/soumitrapy/da6401_assignment2/blob/main/partA/notebook1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# !git clone https://github.com/soumitrapy/da6401_assignment2.git project
# %cd project
# !wget https://storage.googleapis.com/wandb_datasets/nature_12K.zip
# !unzip -q nature_12K.zip -d data/

Cloning into 'project'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 45 (delta 16), reused 36 (delta 13), pack-reused 0 (from 0)
Receiving objects: 100% (45/45), 25.15 KiB | 12.58 MiB/s, done.
Resolving deltas: 100% (16/16), done.
/content/project/project
--2025-04-22 09:47:14--  https://storage.googleapis.com/wandb_datasets/nature_12K.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.98.207, 142.251.107.207, 74.125.196.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.98.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3816687935 (3.6G) [application/zip]
Saving to: ‘nature_12K.zip’

nature_12K.zip      100%[===================>]   3.55G   141MB/s    in 37s     

2025-04-22 09:47:51 (98.1 MB/s) - ‘nature_12K.zip’ saved [3816687935/3816687935]



In [2]:
# !git pull
%cd partA

[Errno 2] No such file or directory: 'partA'
/home/po/Documents/2nd/dl/a2/partA


In [9]:
import wandb
import yaml
import os
from datetime import datetime

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

import torch
from torch.utils.data import Dataset,DataLoader, random_split

from torchvision import transforms
#from torchvision.io import read_image

In [11]:
import yaml
config = yaml.safe_load(open("config/smallcnn.yaml"))

config['dataset']['batch_size']=5
config['train']['epochs']=1
config['use_wandb']=False
config


{'project': 'dla2parta',
 'use_wandb': False,
 'dataset': {'name': 'CustomDataset',
  'path': '../data/inaturalist_12K/',
  'img_size': 256,
  'class_names': ['Plantae',
   'Mammalia',
   'Animalia',
   'Reptilia',
   'Amphibia',
   'Aves',
   'Fungi',
   'Arachnida',
   'Mollusca',
   'Insecta'],
  'batch_size': 5},
 'train': {'epochs': 1, 'val_interval': 1},
 'model': {'name': 'simplecnn',
  'in_channels': 3,
  'num_classes': 10,
  'num_layers': 5,
  'filters': [16, 32, 64, 32, 16],
  'kernel_size': 3,
  'activation': 'relu',
  'dense_neurons': 100}}

In [12]:
import wandb
if config.get('use_wandb',False):
    wandb.login()

### DataLoader Creation

In [13]:
from preprocessing import CustomDataset
from torchvision import transforms
import torch

# Data augmentation and normalization for training
# Just normalization for validation
cfg = config['dataset']

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(cfg['img_size']),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.RandomResizedCrop(cfg['img_size']),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}
splits = ['train', 'val']
datasets = {x:CustomDataset(path= cfg['path']+x,
                            class_names=cfg['class_names'],
                            transform=data_transforms[x])
            for x in splits}

dataloaders = {x: torch.utils.data.DataLoader(datasets[x],
                                              batch_size=cfg['batch_size'],
                                              shuffle=True
                                              )
                for x in splits}

class_names = datasets['train'].class_names
print(f"train len: {len(datasets['train'])}, val len: {len(datasets['val'])}")

# We want to be able to train our model on an `accelerator <https://pytorch.org/docs/stable/torch.html#accelerators>`__
# such as CUDA, MPS, MTIA, or XPU. If the current accelerator is available, we will use it. Otherwise, we use the CPU.

device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
print(f"device = {device}")

train len: 9999, val len: 2000
device = cpu


In [15]:
for x, y in dataloaders['train']:
    print(x.shape,y.shape)
    break

torch.Size([5, 3, 256, 256]) torch.Size([5])


### Sweep Config

In [ ]:
sweep_config = {
    'method': 'random',
    'metric': {
        'name': 'loss',
        'goal': 'minimize'
    },
    'parameters':{
        'filters':[[16,32,64,32,16], [8,16,16,32,64], [64,32,32,16,8]],
        'kernel_size': [3,5,7],
        'dense_neurons': [100, 1000],
        'activation': ['relu', 'gelu', 'silu'],
    }
}
sweep_config

{'method': 'random',
 'metric': {'name': 'loss', 'goal': 'minimize'},
 'parameters': {'filters': [[16, 32, 64, 32, 16],
   [8, 16, 16, 32, 64],
   [64, 32, 32, 16, 8]],
  'kernel_size': [3, 5, 7],
  'dense_neuron': [100, 1000],
  'activation': ['relu', 'gelu', 'silu']}}

In [ ]:
sweep_id = wandb.sweep(sweep_config, project = 'dla2partb')

In [ ]:
from model.smallcnn import SmallCNN

def wandb_train(cfg=None):
    with wandb.init(cfg=cfg):
        cfg = wandb.config

        loss_fn = torch.nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.01)
        model = SmallCNN(cfg)
        train(model=model, optimizer=optimizer, loss_fn=loss_fn, dataloaders=dataloaders,config=config['train'], model_config=config['model'], scheduler = scheduler, device = device, use_wandb = config['use_wandb'])
        

### Model

In [ ]:
wandb.agent(sweep_id, wandb_train, count=7)

torch.Size([5, 10])